# Noise injection into data

This example presents how the the noise can be injected into any part of the dataset: train, test and validation.
There are three types of noise implemented: 


- Gaussian
- Impulsive
- Shot



There are a few parameters which indicate how a given noise behaves:


- *pa* - Fraction of noisy pixels, the number of affected samples is calculated by: floor(n_samples * pa).
- *pb* - Fraction of noisy bands. When established the number of samples that undergo noise injection, for each sample the: floor(n_bands * pb) bands are affected.
- *bc* - Boolean indicating whether the indexes of affected bands, are constant for each sample. When set to: False, different bands can be augmented with noise for each pixel.
- *mean* - Gaussian noise parameter, the mean of the normal distribution.
- *std* - Gaussian noise parameter, standard deviation of the normal distribution.
- *pw* - Impulsive noise parameter, ratio of whitened pixels for the affected set of samples.


In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import os
import shutil
import re

import clize
import mlflow
import tensorflow as tf
from clize.parameters import multi

from scripts import evaluate_model, prepare_data, artifacts_reporter, train_model
from ml_intuition.enums import Splits, Experiment
from ml_intuition.data.io import load_processed_h5
from ml_intuition.data.utils import get_mlflow_artifacts_path, parse_train_size
from ml_intuition.data.loggers import log_params_to_mlflow, log_tags_to_mlflow

Specify path to the `.npy` dataset and ground truth, as well as the output path to store all the artifacts.

In [3]:
DEST_PATH = 'data_noise_injection_results'
DATA_FILE_PATH = os.path.join(os.path.dirname(os.getcwd()), 'datasets/pavia/pavia.npy')
GT_FILE_PAT = os.path.join(os.path.dirname(os.getcwd()), 'datasets/pavia/pavia_gt.npy')
experiment_dest_path = os.path.join(DEST_PATH, 'experiment_0')
os.makedirs(experiment_dest_path, exist_ok=True)

# Prepare the data

To fit into the the pipeline, the data has to be preprocessed. It is achieved by the `prepare_data.main` function. It accepts a path to a `.npy` file with the original cube as well as the corresponding ground truth.  In this example, we randomly extract 250 samples from each class (balanced scenario), use 10% of them as validation set, and extract only spectral information of a pixel. The returned object is a dictionary with three keys: `train`, `test` and `val`. Each of them contains an additional dictionary with `data` and `labels` keys, holding corresponding `numpy.ndarray` objects with the data. For more details about the parameters, refer to the documentation of `prepare_data.main` function (located in `scripts/prepare_data`).

In [4]:
data = prepare_data.main(data_file_path=DATA_FILE_PATH,
                            ground_truth_path=GT_FILE_PAT,
                            output_path=None,
                            train_size=250,
                            val_size=0.1,
                            stratified=True,
                            background_label=0,
                            channels_idx=2,
                            neighborhood_size=None,
                            save_data=False,
                            seed=0)

# Train the model with nosiy training set

The function `trian_model.train` executed the trainig procedure. In order to inject noise into the training set, provide `noise` with a name of the noise type, `noise_sets` with the set you would like to augment, and `noise_params` with the noise parameters. Trained model will be stored under `experiment_dest_path` folder path.

In [5]:
train_model.train(model_name='model_2d',
                    kernel_size=5,
                    n_kernels=200,
                    n_layers=1,
                    dest_path=experiment_dest_path,
                    data=data,
                    sample_size=103,
                    n_classes=9,
                    lr=0.001,
                    batch_size=128,
                    epochs=200,
                    verbose=2,
                    shuffle=True,
                    patience=15,
                    noise=['gaussian'],
                    noise_sets=['train'],
                    noise_params="{\"mean\": 0, \"std\": 1, \"pa\": 0.1, \"pb\": 1}")


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 1, 200)        1200      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 1, 200)        200200    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 1, 200)        200200    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 1, 200)         200200    
_________________________________________________________________
flatten (Flatten)            (None, 1000)              0         
_________________________________________________________________
dense (Dense)                (None, 200)               200200    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               25728     
__________

# Evaluate the model

Evaluate the model, calculating all metrics. All arfticats will be stored under provided `experiment_dest_path`. In this step, it is also possible to inject nosie into the `test` set, similarly to the previous function call.

In [ ]:
evaluate_model.evaluate(
    model_path=os.path.join(experiment_dest_path, 'model_2d'),
    data=data,
    dest_path=experiment_dest_path,
    n_classes=9,
    batch_size=1024,
    noise=['gaussian'],
    noise_sets=['test'],
    noise_params="{\"mean\": 0, \"std\": 1, \"pa\": 0.1, \"pb\": 1}")
tf.keras.backend.clear_session()